In [64]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Replace with your model path
model_path = "/apdcephfs_qy3/share_1594716/yataobian/yang/output/data/general-verifier"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).cuda()

# Example inputs
question = "Where is the capital of China"
ground_truth = "The captical of China is Bejing, the most big city of the world"
student_answer = "it is Peking"

# 创建带决策前缀的prompt（确保模型直接预测Yes/No）
modified_prompt = (
    f"User: ### Question: {question}\n\n"
    f"### Ground Truth Answer: {ground_truth}\n\n"
    f"### Student Answer: {student_answer}\n\n"
    "For the above question, please verify if the student's answer is equivalent to the ground truth answer.\n"
    "Do not solve the question by yourself; just check if the student's answer is equivalent to the ground truth answer.\n"
    "If correct, output \"Final Decision: Yes\". If incorrect, output \"Final Decision: No\".\n"
    "Assistant: Final Decision: "  # 关键修改：将决策部分置于输入序列末尾
)

# 分词处理（注意添加return_offsets_mapping用于定位）
inputs = tokenizer(modified_prompt, 
                  return_tensors="pt", 
                  return_offsets_mapping=True).to(model.device)
input_ids = inputs.input_ids

# 前向推理获取logits
with torch.no_grad():
    outputs = model.forward(input_ids)
logits = outputs.logits  # [batch=1, seq_len=56, vocab_size=32000]

# 确定预测位置（最后一个token的位置）
predict_pos = input_ids.shape[1] - 1  # 对应"Final Decision: "后的预测位置

# 提取目标位置的logits
next_token_logits = logits[0, predict_pos, :]  # [vocab_size]

# 获取Yes/No的token ID（考虑分词细节）
decision_tokens = tokenizer(" Yes", " No", add_special_tokens=False) 
yes_id = decision_tokens.input_ids[0]  # 假设"Yes"为单token
no_id = decision_tokens.input_ids[1]   # 假设"No"为单token

# 计算概率分布
probs = torch.softmax(next_token_logits, dim=0)
yes_prob = probs[yes_id].item()
no_prob = probs[no_id].item()

# 结果判定
print(f"Probabilities - Yes: {yes_prob:.4f}, No: {no_prob:.4f}")
if yes_prob > no_prob:
    print("结论：Final Decision: Yes 概率更高")
else:
    print("结论：Final Decision: No 概率更高")

Probabilities - Yes: 0.0307, No: 0.9692
结论：Final Decision: No 概率更高


In [49]:
inputs

{'input_ids': tensor([[ 1474,    25, 16600, 15846,    25, 10967,   374,   279,  6722,   315,
          5616,   271, 14374, 25413, 29098, 21806,    25,   576,  6427,   938,
           315,  5616,   374,   393, 58852,    11,   279,  1429,  2409,  3283,
           315,   279,  1879,   271, 14374, 11726, 21806,    25,   432,   374,
           393, 58852,   271,  2461,   279,  3403,  3405,    11,  4486, 10146,
           421,   279,  5458,   594,  4226,   374, 13578,   311,   279,  4910,
          8046,  4226,   624,  5404,   537, 11625,   279,  3405,   553,  6133,
            26,  1101,  1779,   421,   279,  5458,   594,  4226,   374, 13578,
           311,   279,  4910,  8046,  4226,   624,  2679,   279,  5458,   594,
          4226,   374,  4396,    11,  2550,   330, 19357, 40425,    25,  7414,
          3263,  1416,   279,  5458,   594,  4226,   374, 15114,    11,  2550,
           330, 19357, 40425,    25,  2308,  3263, 21388,    25, 13023, 40425,
            25,  7414]], device='cuda:

In [52]:
tokenizer.decode(7414)

' Yes'

In [46]:
def check_implication(self, answer1, answer2, question, *args, **kwargs):
    # 创建prompt模板（与原始实现一致）
    prompt = (
        f"User: ### Question: {question}\n\n"
        f"### Ground Truth Answer: {answer1}\n\n"
        f"### Student Answer: {answer2}\n\n"
        "For the above question, please verify if the student's answer is equivalent to the ground truth answer.\n"
        "Do not solve the question by yourself; just check if the student's answer is equivalent to the ground truth answer.\n"
        "If the student's answer is correct, output \"Final Decision: Yes\". If the student's answer is incorrect, output \"Final Decision: No\". Assistant: Final Decision: "
    )

    # Tokenize输入（需要添加return_offsets_mapping获取位置信息）
    inputs = self.tokenizer(prompt, return_tensors="pt", return_offsets_mapping=True).to(self.model.device)
    
    # 前向传播获取logits
    with torch.no_grad():
        outputs = self.model.forward(**inputs)
    
    # 提取最后一个token位置（即"Final Decision: "后的位置）
    last_token_idx = inputs.input_ids.shape[-1] - 1
    next_token_logits = outputs.logits[0, last_token_idx, :]  # [batch, seq_len, vocab]
    
    # 获取"Yes"和"No"的token ID
    yes_token = self.tokenizer.encode("Yes", add_special_tokens=False)[0]
    no_token = self.tokenizer.encode("No", add_special_tokens=False)[0]
    
    # 计算概率分布
    probs = torch.softmax(next_token_logits, dim=-1)
    yes_prob = probs[yes_token].item()
    no_prob = probs[no_token].item()
    
    # 判断概率高低
    return "Final Decision: Yes" if yes_prob > no_prob else "Final Decision: No"